## 3. Temporal Convolutional Network (TCN) Classification

This section implements the Temporal Convolutional Network (TCN) classifier
used in the original study to model temporal dependencies in sequences of
dynamic functional connectivity features.


### Expected input shape

Each subject is represented as a sequence of T dynamic functional
connectivity vectors of dimensionality 6670, i.e., a tensor of shape
(T, 6670), where T is fixed per acquisition site.


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def tcn_residual_block(x, filters=3, kernel_size=3, dilation=1, dropout=0.3, name="res"):
    shortcut = x

    # 1st dilated causal conv
    y = layers.Conv1D(filters, kernel_size, padding="causal",
                      dilation_rate=dilation, use_bias=True,
                      name=f"{name}_conv1")(x)
    y = layers.BatchNormalization(name=f"{name}_bn1")(y)
    y = layers.ReLU(name=f"{name}_relu1")(y)
    y = layers.Dropout(dropout, name=f"{name}_drop1")(y)

    # 2nd dilated causal conv (same dilation)
    y = layers.Conv1D(filters, kernel_size, padding="causal",
                      dilation_rate=dilation, use_bias=True,
                      name=f"{name}_conv2")(y)
    y = layers.BatchNormalization(name=f"{name}_bn2")(y)
    y = layers.ReLU(name=f"{name}_relu2")(y)
    y = layers.Dropout(dropout, name=f"{name}_drop2")(y)

    # Match channels for residual add if needed
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv1D(filters, 1, padding="same", use_bias=True,
                                 name=f"{name}_proj")(shortcut)

    out = layers.Add(name=f"{name}_add")([shortcut, y])
    return out


/Users/mariaborca/Documents/AI_2023-2026/Semestrul_5/KBS/Report_3/adhd-tcn-replication/.venv/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
def tdnet_tcn_classifier(T, F=6670, filters=3, kernel_size=3, dropout=0.3):
    """
    Paper-faithful TDNet temporal module + fusion + classifier.
    Input:  (T, 6670)
    Output: (2,) softmax
    """
    inp = layers.Input(shape=(T, F), name="H_seq")  # H ∈ R^{T×6670}

    # TCN: Q=3 residual blocks, dilations = 1,2,4
    x = inp
    x = tcn_residual_block(x, filters=filters, kernel_size=kernel_size, dilation=1, dropout=dropout, name="tcn_b1")
    x = tcn_residual_block(x, filters=filters, kernel_size=kernel_size, dilation=2, dropout=dropout, name="tcn_b2")
    x = tcn_residual_block(x, filters=filters, kernel_size=kernel_size, dilation=4, dropout=dropout, name="tcn_b3")

    HQ = x  # shape: (B, T, D) where D=filters=3

    # 1×1 conv over HQ with tanh to get P ∈ R^T (per sample)
    P = layers.Conv1D(1, 1, padding="same", activation="tanh", name="time_weight_conv")(HQ)  # (B,T,1)

    # Fuse: H' = P H  -> (B, 6670)
    # reshape P to (B,1,T) and matmul with H (B,T,F) -> (B,1,F) -> squeeze -> (B,F)
    P_row = layers.Permute((2,1), name="P_row")(P)  # (B,1,T)
    H_fused = layers.Lambda(lambda z: tf.matmul(z[0], z[1]), name="fuse_matmul")([P_row, inp])  # (B,1,F)
    H_fused = layers.Lambda(lambda z: tf.squeeze(z, axis=1), name="fuse_squeeze")(H_fused)      # (B,F)

    # Classifier: 512 -> 128 -> 2 (softmax)
    x = layers.Dense(512, activation="relu", name="fc1")(H_fused)
    x = layers.Dense(128, activation="relu", name="fc2")(x)
    out = layers.Dense(2, activation="softmax", name="fc3")(x)

    return Model(inp, out, name=f"TDNet_TCN_T{T}")


In [3]:
model = tdnet_tcn_classifier(T=8)  # example: NYU
model.compile(
    optimizer=tf.keras.optimizers.Adam(),   # default params
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
